In [1]:
import bokeh.io
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import SingleIntervalTicker, LinearAxis, ColumnDataSource, HoverTool
from bokeh.models import LinearAxis, Range1d, Legend
from bokeh.layouts import column, grid
from bokeh.resources import INLINE
bokeh.io.output_notebook(INLINE)
import pandas as pd
import talib

Loading BokehJS ...

#### 透過Bokeh套件繪製互動式圖表
基本上Bokeh提供蠻完整的documentation，花一些時間應該可以較為熟悉使用的方法。

https://bokeh.org

In [2]:
# Interactive Plot簡單範例
def chart(data):
    # get data & reset_index()
    df = data.copy().reset_index(drop = True).reset_index() # -> 生成"index"這個欄位，方便後續x-axis定位
    
    # 讀資料進來時，記得要先生成Date欄位
    df["Date"] = df["Date"].astype(str)
    
    # 準備好主圖需要繪製的均線
    df["MA5"] = df.rolling(5).mean()["Close"].tolist()
    df["MA10"] = df.rolling(10).mean()["Close"].tolist()
    df["MA20"] = df.rolling(20).mean()["Close"].tolist()
    df["MA60"] = df.rolling(60).mean()["Close"].tolist()
    df["MA120"] = df.rolling(120).mean()["Close"].tolist()
    
    # 先準備好子圖想呈現的技術指標，本例為繪製Directional Movement Index - +DI, -DI, ADX
    df["P_DI"] = talib.PLUS_DI(df["High"], df["Low"], df["Close"], timeperiod = 14)
    df["N_DI"] = talib.MINUS_DI(df["High"], df["Low"], df["Close"], timeperiod = 14)
    df["ADX"] = talib.ADX(df["High"], df["Low"], df["Close"], timeperiod = 14)
    
    # inc -> 紅K的部分；dec -> 綠K的部分
    inc = df.Close > df.Open
    dec = df.Open > df.Close
    inc_data = df[inc]
    dec_data = df[dec]
    # Source
    df_source = ColumnDataSource(df)
    inc_source = ColumnDataSource(inc_data)
    dec_source = ColumnDataSource(dec_data)

    # 設定 hover (十字線移動下，碰到K棒會顯示的圖示)
    # (圖例顯示名稱, @欄位名稱)
    hover = HoverTool(
        tooltips = [
            ("Date", "@Date"),
            ("Close", "@Close"),
            ("Open", "@Open"),
            ("High", "@High"),
            ("Low", "@Low"),
            ("Volume", "@TotalVolume"),
        ],
        formatters = {"@Date": "datetime"}
    )
    # (十字線移動下，碰到線會顯示的圖示)
    hover_DMI = HoverTool(
        tooltips=[
            ("Date", "@Date"),
            ("+DI", "@P_DI"),
            ("-DI", "@N_DI"),
            ("ADX", "@ADX")
        ],
        formatters={"@Date": "datetime"}
    )
    
    # 設定圖表
    title = "台指期(TXF) Historical Chart"
    x_end = len(df)
    init = 120 # 一打開想呈現幾根
    x_start = x_end - init
    interval_freq = init / 10 # x-axis間隔頻率
    
    # 設定主圖的y軸range -> problem: 當縮到分K時，candlestick會非常不明顯！解決方法？
    y_start = df["Close"].min() * 0.95
    y_end = df["Close"].max() * 1.05
    
    # 設定子圖的y軸range
    y_start_2 = 0
    y_end_2 = max(df["P_DI"].max(), df["N_DI"].max(), df["ADX"].max()) * 1.05
    
    # 價量與均線圖
    plot1 = figure(plot_width = 1350, title = title, plot_height = 500, x_range = (x_start, x_end), y_range = (y_start, y_end),
                tools = [hover, "pan,zoom_in,zoom_out,crosshair,reset,save"], toolbar_location = "above", y_axis_label = "price")
    # DMI指標線圖
    # 透過設定x_range = plot1.x_range，實現上下二圖可同時左右移動。
    plot2 = figure(plot_width = 1350, title = "DMI Technical Chart", plot_height = 250, x_range = plot1.x_range, y_range = (y_start_2, y_end_2),
                background_fill_color = "#fafafa", tools = [hover_DMI, "pan,zoom_in,zoom_out,crosshair,reset,save"],
                toolbar_location = "above", y_axis_label = "value")

    for fig in [plot1,plot2]:
        fig.title.text_font_size = "16pt"

        # map dataframe indices to date strings and use as label overrides
        fig.xaxis.major_label_overrides = {i: date.strftime("%Y/%m/%d") for i, date in enumerate(pd.to_datetime(df["Date"]))}
        fig.xaxis.ticker = SingleIntervalTicker(interval = interval_freq)

    # 繪製K棒圖
    plot1.segment("index", 'High', "index", "Low", color = "black", source = df_source) # 從documentation看一下segment即知此為繪製上下引線！
    plot1.vbar(x = "index", width = 0.5, bottom = "Open", top = "Close", fill_color = "#cd5c5c", line_color = "black", source = inc_source) # 紅K
    plot1.vbar(x = "index", width = 0.5, top = "Open", bottom = "Close", fill_color = "#2e8b57", line_color = "black", source = dec_source) # 綠K
    
    # 繪製均線圖
    ma_legend_items = []
    for ma_name, color in zip(["MA5", "MA10", "MA20", "MA60", "MA120"], ["deepskyblue", "navajowhite", "slateblue", "pink", "lightgreen"]):
        # color名稱可參考matplotlib color
        ma_df = df[["index", ma_name]]
        source = ColumnDataSource(ma_df)
        ma_line = plot1.line(x = "index", y = ma_name, line_width = 1.5, color = color, alpha = 0.5,
                          muted_color = color, muted_alpha = 0.2, source = source)
        ma_legend_items.append((ma_name, [ma_line]))
    
    # 設定圖例
    legend = Legend(items = ma_legend_items, location = (0, 250))
    plot1.add_layout(legend, "left")

    # 設定另一個y軸scale給Volume
    y2_start = df["TotalVolume"].min() * 0.95
    y2_end = df["TotalVolume"].max() * 1.05
    plot1.extra_y_ranges = {"vol": Range1d(y2_start, y2_end)}
    plot1.vbar(x = "index", width = 0.5, top = "TotalVolume", bottom = 0, y_range_name = "vol", color = "blue", alpha = 0.2, source = df_source)
    plot1.add_layout(LinearAxis(y_range_name = "vol", axis_label = "vol"), "right")

    # DMI Chart
    DMI_legend_items = []
    for index_name, color in zip(["P_DI", "N_DI", "ADX"], ["indianred", "yellowgreen", "mediumpurple"]):
        index_line = plot2.line("index", index_name, line_width = 3, color = color, alpha = 0.8, muted_color = color,
                                 muted_alpha = 0.2, source = df_source)
        DMI_legend_items.append((index_name, [index_line]))
    
    # set DMI legend
    legend = Legend(items = DMI_legend_items, location = (0, 50))
    plot2.add_layout(legend, "left")
    
     # set legend mode
    for fig in [plot1,plot2]:
        # set legend
        fig.legend.label_text_font_size = "8pt"
        # use hide or mute, 用hide的話去點像MA5的那個圖例，MA5會消失；選擇 "mute"的話則是會參照繪製該條線時所設定的參數muted_color & muted_alpha(透明度)
#         fig.legend.click_policy = "hide"
        fig.legend.click_policy = "mute"
        
    # use jupyter output
    output_notebook()
    # show, 透過column排，也可用row之類的
    show(column(plot1, plot2))
    
    # 挑戰：當寫出交易策略之後，針對進出場訊號的部分，能否做到跟multichart一樣顯示在K棒上？(提示：close或open跟訊號(1/0/-1)去做一個相乘, 然後利用bokeh散佈圖的上三角跟下三角型，用線相連？)

In [3]:
data = pd.read_csv("TXF 2010-2020 daily.txt", sep = ",", encoding = "utf-8", engine = "c")
data.Datetime = pd.to_datetime(data.Datetime)
data["Date"] = [i.date() for i in data["Datetime"]]

In [4]:
chart(data)
# MA5, MA10, ......, P_DI, N_DI, ADX是可以點擊的！result: hide or mute

Loading BokehJS ...